In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
%matplotlib qt

In [2]:
import sys
sys.path.append('..')
import neural_ode.NeuralODE

In [3]:
import neural_ode.ODESolvers

In [4]:
tf.keras.backend.set_floatx('float64')

In [5]:
model = keras.Sequential(
    [
        keras.Input(shape=(2,)),
        layers.Dense(2, name="layer1"),
    ]
)


In [6]:
n_ode = neural_ode.NeuralODE.NeuralODE(model, 2, 
                                       solver=neural_ode.ODESolvers.HeunsMethod())

In [7]:
n_ode.model.variables

[<tf.Variable 'layer1/kernel:0' shape=(2, 2) dtype=float64, numpy=
 array([[-0.77324656, -0.38915579],
        [ 0.19777676,  0.79469954]])>,
 <tf.Variable 'layer1/bias:0' shape=(2,) dtype=float64, numpy=array([0., 0.])>]

In [ ]:
n_ode.pretrain(t_scale=1.0, step_size=0.5, n_epoch = 1)

In [ ]:
# test newtons method

In [ ]:
y0 = tf.constant([[3.0, 4.0]], dtype=tf.float64)
resid = lambda y: y*y-tf.constant([[2.0, 3.0]], dtype=tf.float64)
jac = lambda y: tf.eye(2, dtype=tf.float64)*y*2
neural_ode.ODESolvers.newtons_method(y0, resid, jac)

In [ ]:
sol11

In [ ]:
resid(sol11)

In [ ]:
n_ode.model.variables

In [ ]:
# Generate solution

In [8]:
#
N_n = int(2)
c = 0.1
k = 4.0
def oscilator(t, y):
    return np.array([y[1], -c*y[1]-k*y[0]])
t_final = 20.0
n_eval = int(501)
t_span = np.array([0.0, t_final])
y0 = np.array([1.0, 0.0])

In [9]:
sol = solve_ivp(oscilator, t_span, y0, t_eval=np.linspace(0, t_final, num=n_eval))

In [10]:
plt.plot(sol.t, sol.y[0,:])

In [11]:
# transform to tensorflow
t_span_tf = tf.constant(t_span)
y0_tf = tf.constant(y0, dtype=tf.float64)
t_target = tf.constant(sol.t)
y_target = tf.constant(np.transpose(sol.y) )

In [ ]:
sol, _ = n_ode.solver(n_ode.ode_wrap, tf.constant([0.0, 0.1], dtype=tf.float64), tf.expand_dims(y0_tf, axis=0),
                            step_size=0.05, jac = n_ode.grad_inps_wrap)

In [ ]:
plt.plot(sol['t'].numpy(), sol['y'][:,0].numpy())

In [ ]:
sol1 = n_ode.forward_solve(t_target, y_target[0,:])
fig = plt.figure()
ax = plt.gca()
ax.plot(t_target.numpy(), y_target[:,0].numpy())
ax.plot(sol1['t'].numpy(), sol1['y'][:,0].numpy())

In [ ]:
# adjoint method

In [ ]:
loss, dl_dy, a = n_ode.adjoint_method(t_target[0:3], y_target[0:3,:])

In [39]:
n_ode.model.variables[0].assign(np.array([[0.3, -k+1.0], [1.3, -c+0.2]]))
n_ode.model.variables[1].assign(np.array([0.2,0]))

<tf.Variable 'UnreadVariable' shape=(2,) dtype=float64, numpy=array([0.2, 0. ])>

In [ ]:
# fit

In [40]:
n_epoch = 30
n_ode.fit(t_target, y_target, n_epoch=n_epoch, n_fold=10, adjoint_method=False, 
          opt=tf.keras.optimizers.Adam(learning_rate=0.2), conjugate=True)

12
--- Epoch #1 ---
Total loss of epoch: 1.307338409125805
Elapsed time for epoch: 1.2411830425262451
--- Epoch #2 ---
Total loss of epoch: 1.3931036740541458
Elapsed time for epoch: 1.403458833694458
--- Epoch #3 ---
Total loss of epoch: 0.6336121559143066
Elapsed time for epoch: 1.2503232955932617
--- Epoch #4 ---
Total loss of epoch: 0.9058390399441123
Elapsed time for epoch: 1.2418205738067627
--- Epoch #5 ---
Total loss of epoch: 0.4058857779018581
Elapsed time for epoch: 1.2449767589569092
--- Epoch #6 ---
Total loss of epoch: 0.38198162452317774
Elapsed time for epoch: 1.234116792678833
--- Epoch #7 ---
Total loss of epoch: 0.29011498484760523
Elapsed time for epoch: 1.2789292335510254
--- Epoch #8 ---
Total loss of epoch: 0.20313754677772522
Elapsed time for epoch: 1.4017608165740967
--- Epoch #9 ---
Total loss of epoch: 0.23812321806326509
Elapsed time for epoch: 1.3188748359680176
--- Epoch #10 ---
Total loss of epoch: 0.18918147933436558
Elapsed time for epoch: 1.30620837211

In [38]:
n_epoch = 30
n_ode.fit(t_target, y_target, n_epoch=n_epoch, n_fold=10, adjoint_method=False, 
          opt=tf.keras.optimizers.SGD(learning_rate=0.8), conjugate=True)

12
--- Epoch #1 ---
Total loss of epoch: 0.963888140860945
Elapsed time for epoch: 1.3265092372894287
--- Epoch #2 ---
Total loss of epoch: 0.32587061217054725
Elapsed time for epoch: 1.3407642841339111
--- Epoch #3 ---
Total loss of epoch: 0.12520851369481534
Elapsed time for epoch: 1.3659987449645996
--- Epoch #4 ---
Total loss of epoch: 0.0724606157746166
Elapsed time for epoch: 1.4257349967956543
--- Epoch #5 ---
Total loss of epoch: 0.03744320513214916
Elapsed time for epoch: 1.307999610900879
--- Epoch #6 ---
Total loss of epoch: 0.023122608254197985
Elapsed time for epoch: 1.3046317100524902
--- Epoch #7 ---
Total loss of epoch: 0.012711232528090477
Elapsed time for epoch: 1.4035277366638184
--- Epoch #8 ---
Total loss of epoch: 0.008447044572676532
Elapsed time for epoch: 1.3032073974609375
--- Epoch #9 ---
Total loss of epoch: 0.005641877360176295
Elapsed time for epoch: 1.3879101276397705
--- Epoch #10 ---
Total loss of epoch: 0.005621706921374425
Elapsed time for epoch: 1.38

In [ ]:
# Check derivatives

In [ ]:
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.0, -c]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
loss, dl_dp = n_ode.usual_method(t_target, y_target)
#loss, dL_dy, a = n_ode.adjoint_method(t_target, y_target)
#dl_dp = a[2:]

In [ ]:
dp = 0.00001
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c+dp]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
loss2, dl_dp2 = n_ode.usual_method(t_target, y_target)

In [ ]:
(loss2-loss)/dp

In [ ]:
dl_dp